In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime

#from func_data_prep import process_string, nutri_macro_vars, personal_vars, amount_vars, nutri_micro_vars, nutri_cfg_vars, nutri_fg_vars

plt.rcParams['font.family'] = 'Arial'

#from pingouin import partial_corr

#from tqdm import tqdm

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
"""
meta = pd.read_csv(
    "/Users/rohansingh/WORK/qiime2_2024/metadata_files/fay_meta_may24.csv", index_col=0
)
meta.rename(columns={"subject_app_key": "subject_key"}, inplace=True)

if "age_group_2" not in meta.columns:
    meta["age_group_2"] = pd.cut(
        meta["age"], bins=[0, 35, 50, np.inf], labels=["<35", "35-50", ">50"]
    )
"""

'\nmeta = pd.read_csv(\n    "/Users/rohansingh/WORK/qiime2_2024/metadata_files/fay_meta_may24.csv", index_col=0\n)\nmeta.rename(columns={"subject_app_key": "subject_key"}, inplace=True)\n\nif "age_group_2" not in meta.columns:\n    meta["age_group_2"] = pd.cut(\n        meta["age"], bins=[0, 35, 50, np.inf], labels=["<35", "35-50", ">50"]\n    )\n'

In [4]:

df_food = pd.read_csv("df_food_filtered_with_food_group.csv",index_col=0)
df_food_bkp = df_food.copy()

df_food = df_food[df_food["eaten_quantity_in_gram"] > 0]
df_food = df_food[df_food["energy_kcal_eaten"] > 0]

df_food.columns

C:\Users\Sophie\AppData\Local\Temp\ipykernel_3252\248216210.py:1: DtypeWarning: Columns (2,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food = pd.read_csv("df_food_filtered_with_food_group.csv",index_col=0)


Index(['food_id', 'barcode', 'dish_id', 'eaten_quantity', 'eaten_unit',
       'subject_key', 'eaten_at', 'eaten_at_utc_offset', 'media_count',
       'food_group_cname', 'type', 'display_name_en', 'display_name_fr',
       'display_name_de', 'fallback_food_id', 'standard_portion_quantity',
       'standard_portion_unit', 'specific_gravity', 'alcohol',
       'all_trans_retinol_equivalents_activity', 'beta_carotene',
       'beta_carotene_activity', 'calcium', 'carbohydrates', 'chloride',
       'cholesterol', 'energy_kcal', 'energy_kj', 'fat',
       'fatty_acids_monounsaturated', 'fatty_acids_polyunsaturated',
       'fatty_acids_saturated', 'fiber', 'folate', 'iodide', 'iron',
       'magnesium', 'niacin', 'pantothenic_acid', 'phosphorus', 'potassium',
       'protein', 'salt', 'sodium', 'starch', 'sugar', 'vitamin_a_activity',
       'vitamin_b1', 'vitamin_b12', 'vitamin_b2', 'vitamin_b6', 'vitamin_c',
       'vitamin_d', 'vitamin_e_activity', 'water', 'zinc',
       'eaten_quantit

In [5]:
nutrient_units = {
    "sugar": "g",
    "fatty_acids_saturated": "g",
    "salt": "g",
    "fatty_acids_monounsaturated": "g",
    "fatty_acids_polyunsaturated": "g",
    "cholesterol": "mg",
    # "vitamin_a": "IU",
    "all_trans_retinol_equivalents_activity": "IU",
    "vitamin_c": "mg",
    "beta_carotene": "mcg",
    # "vitamin_e": "TAE",
    "vitamin_e_activity": "TAE",
    "vitamin_d": "mcg",
    # "vitamin_k": "mcg",
    # "thiamin": "mg",
    # "riboflavin": "mg",
    "niacin": "mg",
    "vitamin_b6": "mg",
    "folate": "mcg",
    "vitamin_b12": "mcg",
    "calcium": "mg",
    "phosphorus": "mg",
    "magnesium": "mg",
    "iron": "mg",
    "zinc": "mg",
    # "copper": "mg",
    # "selenium": "mcg",
    "potassium": "mg",
    "sodium": "mg",
    # "caffeine": "mg",
    # "theobromine": "mg",
    "pantothenic_acid": "mg",
    "vitamin_b1": "mg",
    "vitamin_b2": "mg",
}

conversion_factors = {
    "mg": 1000,
    "g": 1,
    "mcg": 1000000,
    "IU": 1,
    "TAE": 1,
}

In [6]:
def scale_for_nbc(df, nutrient_cols, scaling_factor=2000):
    df = df.copy()

    for nutr in nutrient_cols:
        conversion_factor = conversion_factors[nutrient_units[nutr]]
        # df_food[nutr + "_eaten"].fillna(0, inplace=True)

        df[nutr + "_scaled"] = (
            df[nutr + "_eaten"]
            * (scaling_factor / df["energy_kcal_eaten"])
            * conversion_factor
        )

    return df


t = scale_for_nbc(df_food.fillna(0), list(nutrient_units.keys()), scaling_factor=2000)

In [7]:
t[[i for i in t.columns if "scaled" in i]].columns

Index(['sugar_scaled', 'fatty_acids_saturated_scaled', 'salt_scaled',
       'fatty_acids_monounsaturated_scaled',
       'fatty_acids_polyunsaturated_scaled', 'cholesterol_scaled',
       'all_trans_retinol_equivalents_activity_scaled', 'vitamin_c_scaled',
       'beta_carotene_scaled', 'vitamin_e_activity_scaled', 'vitamin_d_scaled',
       'niacin_scaled', 'vitamin_b6_scaled', 'folate_scaled',
       'vitamin_b12_scaled', 'calcium_scaled', 'phosphorus_scaled',
       'magnesium_scaled', 'iron_scaled', 'zinc_scaled', 'potassium_scaled',
       'sodium_scaled', 'pantothenic_acid_scaled', 'vitamin_b1_scaled',
       'vitamin_b2_scaled'],
      dtype='object')

In [8]:
t[t["iron_scaled"] > 18].drop_duplicates("combined_name")["combined_name"]

0         Lamb, filet, pan fried, medium, without additi...
66                                 Anchovy, in oil, drained
114       Berries, (average), cooked, without addition o...
126             Beetroot, steamed, without addition of salt
1014      Beef, entrecote, pan fried, medium, without ad...
                                ...                        
514279           Planted. - Chicken Curry, 100% plant-based
514401                                       CHOCOLATE CHIP
514717                                          PROTEIN BAR
514877                          Beavita - Shake (Chocolate)
514934                                             ovosport
Name: combined_name, Length: 1037, dtype: object

In [9]:
t[t["folate_scaled"] > 400].drop_duplicates("combined_name")["combined_name"]

114       Berries, (average), cooked, without addition o...
126             Beetroot, steamed, without addition of salt
1010                                       Beaten egg white
1066            Broccoli, steamed, without addition of salt
1529              Carrot, steamed, without addition of salt
                                ...                        
513989                                     Ovomaltine Drink
514195                                Vitamin Well - Reload
514324                      Ovomaltine Drink - High Protein
514877                          Beavita - Shake (Chocolate)
514934                                             ovosport
Name: combined_name, Length: 687, dtype: object

In [10]:
dri_dict = {
    "sugar_scaled": 125,
    "fatty_acids_saturated_scaled": 22,
    "salt_scaled": 6,
    "fatty_acids_monounsaturated_scaled": None,
    "fatty_acids_polyunsaturated_scaled": 12,
    "cholesterol_scaled": 300,
    "all_trans_retinol_equivalents_activity_scaled": 700,
    "vitamin_c_scaled": 75,
    "beta_carotene_scaled": None,
    "vitamin_e_activity_scaled": 15,
    "vitamin_d_scaled": 7.5,
    "niacin_scaled": 14,
    "vitamin_b6_scaled": 1.3,
    "folate_scaled": 400,
    "vitamin_b12_scaled": 2.4,
    "calcium_scaled": 1000,
    "phosphorus_scaled": 700,
    "magnesium_scaled": 320,
    "iron_scaled": 18,
    "zinc_scaled": 8,
    "potassium_scaled": 4700,
    "sodium_scaled": 2400,
}


# def calculate_qi_metric(row, nutrient_cols):

#     qi_ratios = []
#     for nutr in nutrient_cols:
#         qi_ratios.append(row[nutr] / dri_dict[nutr])

#     return np.mean(qi_ratios)

In [11]:
def calculate_qi_ratios(row, nutrient_cols):

    qi_ratios = {}
    for nutr in nutrient_cols:
        qi_ratios[nutr + "_ratio"] = row[nutr] / dri_dict[nutr]

    qi_ratios["qi_metric"] = np.sum([i for i in qi_ratios.values()]) / len(
        nutrient_cols
    )

    qi_ratios["combined_name"] = row["combined_name"]
    qi_ratios["food_group"] = row["food_group"]
    qi_ratios["energy_kcal_eaten"] = row["energy_kcal_eaten"]
    qi_ratios["eaten_quantity_in_gram"] = row["eaten_quantity_in_gram"]

    return qi_ratios

In [12]:
qi_nutrients = [
    "all_trans_retinol_equivalents_activity_scaled",
    "vitamin_c_scaled",
    "vitamin_e_activity_scaled",
    "vitamin_d_scaled",
    "niacin_scaled",
    "vitamin_b6_scaled",
    "folate_scaled",
    "vitamin_b12_scaled",
    "calcium_scaled",
    "phosphorus_scaled",
    "magnesium_scaled",
    "iron_scaled",
    "zinc_scaled",
    "potassium_scaled",
    # "fatty_acids_polyunsaturated_scaled",
]

In [13]:
qi_ratios = t.apply(calculate_qi_ratios, axis=1, nutrient_cols=qi_nutrients)

qi_ratios = qi_ratios.apply(pd.Series)

qi_ratios

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,combined_name,food_group,energy_kcal_eaten,eaten_quantity_in_gram
0,0.230105,0.000000,0.178971,0.000000,5.848514,1.445534,0.872483,11.744966,0.187919,4.218600,1.300336,1.789709,7.214765,0.856776,2.563477,"Lamb, filet, pan fried, medium, without additi...",meat_fish_seafood,163.90,110.0
3,0.230105,0.000000,0.178971,0.000000,5.848514,1.445534,0.872483,11.744966,0.187919,4.218600,1.300336,1.789709,7.214765,0.856776,2.563477,"Lamb, filet, pan fried, medium, without additi...",meat_fish_seafood,149.00,100.0
4,0.000000,4.705882,0.261438,0.000000,0.756303,2.413273,0.686275,0.000000,0.549020,0.616246,1.838235,0.653595,0.490196,1.084689,1.003939,"Pineapple, unsweetened, canned",fruits,17.85,35.0
5,0.000000,4.705882,0.261438,0.000000,0.756303,2.413273,0.686275,0.000000,0.549020,0.616246,1.838235,0.653595,0.490196,1.084689,1.003939,"Pineapple, unsweetened, canned",fruits,17.85,35.0
6,0.000000,4.705882,0.261438,0.000000,0.756303,2.413273,0.686275,0.000000,0.549020,0.616246,1.838235,0.653595,0.490196,1.084689,1.003939,"Pineapple, unsweetened, canned",fruits,20.40,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514955,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Alnatura - Kakao Knusper Reis,sweets_desserts,60.64,16.0
514956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Alnatura - Kakao Knusper Reis,sweets_desserts,60.64,16.0
514957,0.000000,0.000000,0.000000,2.739726,0.000000,0.000000,0.000000,4.337900,3.287671,0.000000,0.000000,0.000000,0.000000,0.000000,0.740378,alpro - Joghurt (Heidelbeere),yogurt_fresh_dairy,73.00,100.0
514958,0.000000,14.545455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.038961,Biotta Sauerkraut,vegetable_fruit_juices,22.00,100.0


In [14]:
qi_ratios.describe()

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,energy_kcal_eaten,eaten_quantity_in_gram
count,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000,317780.000000
mean,0.400287,4.261686,0.972179,0.386326,1.716105,3.546673,1.336722,1.550485,1.261794,1.665848,1.551587,0.931202,1.059263,1.170699,1.557918,105.359098,79.673623
std,3.515382,14.318611,4.407516,2.763487,12.708898,34.987105,4.324491,16.543500,2.493453,2.574719,6.480116,15.977264,3.389862,3.372541,5.528555,117.319215,83.585647
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006700,0.020000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.520000,22.000000
50%,0.000000,0.000000,0.132231,0.000000,0.250289,0.433369,0.150177,0.000000,0.181818,0.873016,0.489130,0.111390,0.520833,0.272777,0.628261,74.600000,50.000000
75%,0.000000,0.993842,0.629630,0.000000,1.666667,1.639344,0.716667,0.000000,1.192547,3.000000,1.539855,0.854701,1.428571,1.319149,1.745608,144.000000,100.000000
max,242.160279,220.994475,350.877193,108.096871,962.406015,1709.401709,247.368421,2380.952381,126.315789,177.443609,641.447368,1666.666667,263.157895,304.591265,369.537613,2797.500000,3000.000000


In [15]:
qi_ratios[
    qi_ratios["combined_name"] == "Connie's Kitchen - Jalapeño Ketchup"
].drop_duplicates()

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,combined_name,food_group,energy_kcal_eaten,eaten_quantity_in_gram
503267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Connie's Kitchen - Jalapeño Ketchup,soups_sauces_condiments,39.6,44.0


In [16]:
qi_ratios.groupby("combined_name")["qi_metric"].median().sort_values(
    ascending=False
).sample(10)

combined_name
Daura Damm - Gluten-Free Lager Beer                       0.000000
Coop Betty Bossi Steinofen Pizza belegt mit Ziegenkäse    0.000000
Gemüsebeignet, Gemüsekrapfen                              0.930043
ADEZ nourish your potential - Chilling Coconut            0.000000
MClassic - Schweizer Eier                                 1.900677
Coop : Qualité&Prix : Beerenmischung                      0.000000
Fiorini - Ravioli pur Boeuf                               0.000000
Lindt Lindor Extra Dunkel                                 0.340049
Beetroot falafel Karma Coop - Vegan                       0.000000
Branche Classic - Milchschokolade                         0.000000
Name: qi_metric, dtype: float64

In [17]:
df_food["food_group"].value_counts()

food_group
vegetables                       47567
sweets_desserts                  30641
bread_pastries                   25817
soups_sauces_condiments          22818
beverages                        22752
fruits                           21730
processed_foods                  11330
butter_margarine_spreads         10517
alcoholic_beverages              10505
cereal_grains_starches            9849
nuts_seeds                        9736
hard_semi_hard_cheese             8957
meat_fish_seafood                 8945
others                            8562
milk                              7394
sweetened_beverages               7310
processed_meats                   6720
cream_based_foods                 5822
yogurt_fresh_dairy                5784
soft_cheese                       5558
potatoes                          4987
eggs                              4159
breakfast_cereals                 3905
pulses                            3763
vegetable_fruit_juices            3541
vegetable_oils

In [18]:
qi_ratios[qi_ratios["food_group"] == "milk"].describe()

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,energy_kcal_eaten,eaten_quantity_in_gram
count,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000,7394.000000
mean,0.927015,0.207940,0.471754,0.378494,0.147126,0.399098,0.311858,0.974813,2.388794,2.392003,0.584696,0.042008,0.885498,0.631633,0.767338,66.747878,103.417732
std,0.953565,0.203087,1.041446,2.780186,0.156931,0.480906,0.247401,1.508140,2.204304,2.129300,0.536471,0.230828,0.832127,0.528670,0.655991,66.266142,85.159220
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067500,0.300000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046948,0.000000,0.000000,0.000000,0.000000,0.000000,0.243002,21.300000,40.000000
50%,0.086580,0.327485,0.215686,0.392157,0.210084,0.539811,0.367647,1.470588,3.529412,3.823529,0.919118,0.000000,1.470588,1.001252,1.174696,42.600000,100.000000
75%,1.932773,0.392157,0.215686,0.392157,0.210084,0.678733,0.367647,1.470588,3.529412,3.865546,0.919118,0.000000,1.470588,1.001252,1.174696,92.800000,150.000000
max,3.675676,1.178947,18.461538,102.564103,1.098901,4.895105,0.968182,24.358974,18.461538,12.355212,3.030303,3.780069,5.405405,2.063185,11.703297,680.000000,1000.000000


In [19]:
qi_ratios[qi_ratios["combined_name"] == "Milk"].describe().round(3)

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,energy_kcal_eaten,eaten_quantity_in_gram
count,3020.000,3020.000,3020.000,3020.000,3020.00,3020.000,3020.000,3020.000,3020.000,3020.000,3020.000,3020.0,3020.000,3020.000,3020.000,3020.000,3020.000
mean,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,58.475,85.992
std,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,53.019,77.969
min,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,0.340,0.500
25%,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,20.400,30.000
50%,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,34.000,50.000
75%,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,81.600,120.000
max,1.933,0.392,0.216,0.392,0.21,0.679,0.368,1.471,3.529,3.866,0.919,0.0,1.471,1.001,1.175,680.000,1000.000


In [20]:
qi_ratios[qi_ratios["food_group"] == "milk"]["combined_name"].value_counts()

combined_name
Milk                                              3020
Oat milk                                           901
Soya drink (soy milk)                              633
Coconut milk                                       421
Almond drink (almond milk)                         263
                                                  ... 
ADEZ nourish your potential - Chilling Coconut       1
Coco Coyog Nature aha!                               1
Sojade - So soja! (Raspberry-Passion fruit)          1
Provamel - Organic unsweetened almond drink          1
Migros Bio - Past Vollmilch                          1
Name: count, Length: 246, dtype: int64

In [21]:
qi_ratios[qi_ratios["combined_name"] == "Oat milk"].describe().round(3)

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,energy_kcal_eaten,eaten_quantity_in_gram
count,901.0,901.0,901.000,901.0,901.0,901.0,901.00,901.0,901.000,901.000,901.000,901.000,901.0,901.00,901.000,901.000,901.000
mean,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,39.334,92.332
std,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000,0.000,0.000,0.000,0.0,0.00,0.000,30.734,72.147
min,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,0.213,0.500
25%,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,21.300,50.000
50%,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,21.300,50.000
75%,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,42.600,100.000
max,0.0,0.0,1.158,0.0,0.0,0.0,0.75,0.0,0.047,0.738,0.337,0.052,0.0,0.32,0.243,213.000,500.000


In [22]:
t[t["combined_name"] == "Red Bull Zero Calories"][
    ["niacin_eaten", "vitamin_b6", "vitamin_b12", "energy_kcal_eaten"]
]

,niacin_eaten,vitamin_b6,vitamin_b12,energy_kcal_eaten
513972,0.02,2.0,2.0,4.5
513973,0.02,2.0,2.0,4.5
513974,0.02,2.0,2.0,4.5
513975,0.02,2.0,2.0,4.5
513976,0.02,2.0,2.0,4.5
513977,0.02,2.0,2.0,4.5
513978,0.02,2.0,2.0,4.5
513979,0.02,2.0,2.0,4.5
513980,0.02,2.0,2.0,4.5
513981,0.02,2.0,2.0,4.5


In [23]:
t[t["combined_name"] == "Red Bull Zero Calories"][
    [i for i in t.columns if "scale" in i]
].drop_duplicates().T

,513972
sugar_scaled,0.000000
fatty_acids_saturated_scaled,0.000000
salt_scaled,66.666667
fatty_acids_monounsaturated_scaled,0.000000
fatty_acids_polyunsaturated_scaled,0.000000
cholesterol_scaled,0.000000
all_trans_retinol_equivalents_activity_scaled,0.000000
vitamin_c_scaled,0.000000
beta_carotene_scaled,0.000000
vitamin_e_activity_scaled,0.000000


In [24]:
qi_ratios[qi_ratios["combined_name"].str.contains("Chicken")].drop_duplicates(
    "combined_name"
)

,all_trans_retinol_equivalents_activity_scaled_ratio,vitamin_c_scaled_ratio,vitamin_e_activity_scaled_ratio,vitamin_d_scaled_ratio,niacin_scaled_ratio,vitamin_b6_scaled_ratio,folate_scaled_ratio,vitamin_b12_scaled_ratio,calcium_scaled_ratio,phosphorus_scaled_ratio,magnesium_scaled_ratio,iron_scaled_ratio,zinc_scaled_ratio,potassium_scaled_ratio,qi_metric,combined_name,food_group,energy_kcal_eaten,eaten_quantity_in_gram
192627,0.227434,0.092869,0.451078,2.122720,1.137171,5.587447,0.323383,2.197347,0.082587,3.553660,0.901741,0.442233,1.368159,0.846830,1.381047,"Chicken, whole",meat_fish_seafood,221.1,110.0
192695,0.227434,0.092869,0.451078,2.122720,1.137171,5.587447,0.323383,2.197347,0.082587,3.553660,0.901741,0.442233,1.368159,0.846830,1.381047,Chicken,meat_fish_seafood,201.0,100.0
193993,0.130862,0.000000,0.396947,0.000000,11.995638,4.227833,0.213740,2.099237,0.274809,4.798255,1.288168,0.424088,1.526718,1.201884,2.041298,"Chicken, breast",meat_fish_seafood,235.8,180.0
194586,0.111111,0.000000,0.488889,2.222222,0.000000,2.564103,0.222222,1.620370,0.078889,3.174603,0.729167,0.370370,1.666667,0.567376,0.986856,"Chicken, for fondue",meat_fish_seafood,90.0,50.0
194599,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"Chicken, wing",meat_fish_seafood,283.8,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514279,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.144033,0.000000,0.000000,0.000000,2.812071,0.000000,0.000000,0.568293,"Planted. - Chicken Curry, 100% plant-based",soy_granules_textured_protein,137.7,85.0
514288,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Butter Chicken with Masala Pualo,processed_foods,292.0,200.0
514532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Anna's thai red Chicken-Curry,processed_foods,664.0,400.0
514842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Migrolino - Sandwich (Chicken Cocktail),others,630.0,175.0


In [25]:
t.groupby("combined_name")["qi_metric"].std().round(2).fillna(0)[
    t.groupby("combined_name")["qi_metric"].std().round(2).fillna(0) > 0
].sort_values(ascending=False)

KeyError: 'Column not found: qi_metric'

In [105]:
t[t["combined_name"] == "Andros - Apfel"]

,food_id,barcode,dish_id,eaten_quantity,eaten_unit,subject_key,eaten_at,eaten_at_utc_offset,media_count,food_group_cname,...,folate_scaled,vitamin_b12_scaled,calcium_scaled,phosphorus_scaled,magnesium_scaled,iron_scaled,zinc_scaled,potassium_scaled,sodium_scaled,qi_metric
466020,5244,3608580881865,26635,100.0,g,vfygbe,2019-04-29 11:31:24,120,0,sweets_salty_snacks_alcohol,...,254.901961,0.0,188.235294,352.941176,156.862745,7.843137,3.921569,4313.725490,156.862745,0.525369
466021,5244,3608580881865,39823,100.0,g,b6qjbv,2019-06-15 18:14:52,120,0,sweets_salty_snacks_alcohol,...,254.901961,0.0,188.235294,352.941176,156.862745,7.843137,3.921569,4313.725490,156.862745,0.525369
479107,7019,3608580867968,140222,90.0,g,8bztuh,2020-08-13 10:28:01,120,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479108,7019,3608580867968,140859,90.0,g,8bztuh,2020-08-17 08:11:13,120,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479109,7019,3608580867968,139916,90.0,g,8bztuh,2020-08-11 14:51:49,120,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479110,7019,3608580867968,401361,90.0,g,pyvpkd,2022-11-18 11:25:57,60,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479111,7019,3608580867968,400024,90.0,g,pyvpkd,2022-11-15 08:43:49,60,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479112,7019,3608580867968,54616,90.0,g,6zkdx5,2019-08-12 10:58:28,120,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833
479113,7019,3608580867968,402051,90.0,g,pyvpkd,2022-11-20 10:15:59,60,0,sweets_salty_snacks_alcohol,...,99.148936,0.0,188.936170,226.808511,216.595745,4.680851,1063.829787,4425.531915,0.000000,68.557833


In [178]:
df_food.groupby(["subject_key", "eaten_at"])["combined_name"].value_counts()

subject_key   eaten_at             combined_name                                  
02ae3856ca04  2018-11-26 07:20:04  Coffee, with caffeine                              1
              2018-11-26 07:47:34  Corn Flakes                                        1
                                   Honey n.s.                                         1
                                   Yaourt, yahourt, yogourt ou yoghourt, natural      1
              2018-11-26 08:56:24  Banana                                             1
                                                                                     ..
zznk4w        2022-09-25 06:11:19  Tomato, raw                                        1
              2022-09-25 09:19:35  Coffee, with caffeine                              1
                                   Oat milk                                           1
              2022-09-25 10:24:40  Porridge, prepared, with partially skimmed milk    1
              2022-09-25 10:36:35  Ca